In [1]:
import torch
import numpy as np
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN as GCN_DR
from deeprobust.graph.targeted_attack import Nettack

In [2]:
from greedy_mcmc_attack import *
from collections import defaultdict
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
cora_dataset = Planetoid(root='', name='Citeseer')
data = cora_dataset[0].to(device)
print(data)
idx_train, idx_val, idx_test = data.train_mask.nonzero(as_tuple=True)[0].tolist(), data.val_mask.nonzero(as_tuple=True)[0].tolist(), data.test_mask.nonzero(as_tuple=True)[0].tolist()

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [4]:
data_dr = Dataset(root='/tmp/', name='citeseer', setting='nettack')
adj, features, labels = data_dr.adj, data_dr.features, data_dr.labels
# idx_train, idx_val, idx_test = data_dr.idx_train, data_dr.idx_val, data_dr.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)

Loading citeseer dataset...
Selecting 1 largest connected components


In [5]:
degrees = np.array(adj.sum(axis=1)).flatten()
max_degree_vertex = degrees.argmax()

In [6]:
surrogate = GCN_DR(nfeat=features.shape[1], nclass=labels.max().item()+1, nhid=16,
                with_relu=False, device=device)
surrogate = surrogate.to(device)
surrogate.fit(features, adj, labels, idx_train)

In [7]:
ptb_rate = 0.1

In [8]:
model = Nettack(model=surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
model = model.to(device)
perturbations = int(ptb_rate * (adj.sum() // 2))
model.attack(features, adj, labels, max_degree_vertex, n_perturbations=perturbations)
modified_adj = model.modified_adj

##### Starting attack #####
##### Attack only using structure perturbations #####
##### Attacking the node directly #####
##### Performing 366 perturbations #####
##### ...1/366 perturbations ... #####


/home/wcorcoran/anaconda3/envs/mesp/lib/python3.8/site-packages/numba/core/ir_utils.py:2172: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected set' found for argument 'edges_set' of function 'compute_new_a_hat_uv'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../anaconda3/envs/mesp/lib/python3.8/site-packages/deeprobust/graph/targeted_attack/nettack.py", line 501:
@jit(nopython=True)
def compute_new_a_hat_uv(edge_ixs, node_nb_ixs, edges_set, twohop_ixs, values_before, degs, potential_edges, u):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


##### ...2/366 perturbations ... #####
##### ...3/366 perturbations ... #####
##### ...4/366 perturbations ... #####
##### ...5/366 perturbations ... #####
##### ...6/366 perturbations ... #####
##### ...7/366 perturbations ... #####
##### ...8/366 perturbations ... #####
##### ...9/366 perturbations ... #####
##### ...10/366 perturbations ... #####
##### ...11/366 perturbations ... #####
##### ...12/366 perturbations ... #####
##### ...13/366 perturbations ... #####
##### ...14/366 perturbations ... #####
##### ...15/366 perturbations ... #####
##### ...16/366 perturbations ... #####
##### ...17/366 perturbations ... #####
##### ...18/366 perturbations ... #####
##### ...19/366 perturbations ... #####
##### ...20/366 perturbations ... #####
##### ...21/366 perturbations ... #####
##### ...22/366 perturbations ... #####
##### ...23/366 perturbations ... #####
##### ...24/366 perturbations ... #####
##### ...25/366 perturbations ... #####
##### ...26/366 perturbations ... #####
##### ..

### arch tests

In [50]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [51]:
model_save_path = "../models/citeseer_gcn_model.pth"
list_save_path = "../attacks/citeseer_gcn_edges.pth"

In [52]:
model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, model, device)

In [53]:
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.69
Initial Loss: 1.2167096138000488


In [54]:
from scipy.sparse import csr_matrix

adj1 = data_dr.adj.tocsr()
adj2 = modified_adj.tocsr()

arr_diff = (adj1 - adj2).tocoo()

In [55]:
arr_diff

<2110x2110 sparse matrix of type '<class 'numpy.float32'>'
	with 732 stored elements in COOrdinate format>

In [56]:
data_modded = copy.deepcopy(data)

In [57]:
# thanks gpt
existing = data_modded.edge_index.t().cpu().numpy()

diff_coo = arr_diff.tocoo()
diff_edges = np.stack([diff_coo.row, diff_coo.col], axis=1)

existing_set = set(map(tuple, existing))
diff_set = set(map(tuple, diff_edges))
diff_set_full = diff_set | set((v, u) for u, v in diff_set)

cleaned_edges = existing_set - diff_set_full
print(len(cleaned_edges))

added_edges = diff_set_full - existing_set
print(len(added_edges))

final_edges = cleaned_edges | added_edges

final_edges = torch.tensor(list(final_edges), dtype=torch.long).t().to(device)
data_modded.edge_index = final_edges

9104
732


In [58]:
new_loss, new_accuracy = train.test(data_modded)

print(f"Evasion Change in Accuracy: {(100 * ((new_accuracy - initial_accuracy) / initial_accuracy)):.4f}%")
print(f"Evasion Change in Loss: {(100 * ((new_loss - initial_loss) / initial_loss)):.4f}%")

### poisoning
model_poisoning = GCN(data_modded.x.shape[1], cora_dataset.num_classes, [16]).to(device)
model_poisoning.reset_parameters()
train_modded = Trainable(model_poisoning)
train_modded.fit(data_modded, 1000)

poisoning_loss, poisoning_accuracy = train_modded.test(data_modded)
print(f"Poisoning Change in Accuracy: {(100 * ((poisoning_accuracy - initial_accuracy) / initial_accuracy)):.4f}%")
print(f"Poisoning Change in Loss: {(100 * ((poisoning_loss - initial_loss) / initial_loss)):.4f}%")

Evasion Change in Accuracy: 0.0000%
Evasion Change in Loss: -1.9251%
Epoch 0, Train Loss - 1.792391300201416, Val Loss - 1.7295587062835693, Val Accuracy - 0.448
Epoch 20, Train Loss - 0.15240797400474548, Val Loss - 1.1294240951538086, Val Accuracy - 0.646
Epoch 40, Train Loss - 0.07772330939769745, Val Loss - 1.179220199584961, Val Accuracy - 0.64
Epoch 60, Train Loss - 0.048672474920749664, Val Loss - 1.2037588357925415, Val Accuracy - 0.646
Epoch 80, Train Loss - 0.0421464778482914, Val Loss - 1.1544861793518066, Val Accuracy - 0.66
Epoch 100, Train Loss - 0.041596509516239166, Val Loss - 1.191360592842102, Val Accuracy - 0.652
Epoch 120, Train Loss - 0.056029241532087326, Val Loss - 1.1886845827102661, Val Accuracy - 0.658
Epoch 140, Train Loss - 0.03486502543091774, Val Loss - 1.2094087600708008, Val Accuracy - 0.672
Epoch 160, Train Loss - 0.04513043165206909, Val Loss - 1.2036957740783691, Val Accuracy - 0.658
Epoch 180, Train Loss - 0.031188927590847015, Val Loss - 1.217775344

In [48]:
initial_accuracy

0.69

In [49]:
poisoning_accuracy

0.688